# ERCOT Grid Analytics for Battery Storage & Retail Electricity

Analyze ERCOT data for:
- **Battery Storage** - Arbitrage opportunities, price spreads, optimal dispatch
- **Retail Providers** - Load patterns, price volatility, renewable integration

## Setup


In [1]:
import os
from datetime import datetime
from pathlib import Path

import pandas as pd
import plotly.graph_objects as go
from dotenv import load_dotenv
from plotly.subplots import make_subplots

from tinygrid import ERCOT, ERCOTAuth, ERCOTAuthConfig

# Load credentials
load_dotenv(Path.cwd() / ".env")

auth = ERCOTAuth(ERCOTAuthConfig(
    username=os.getenv("ERCOT_USERNAME"),
    password=os.getenv("ERCOT_PASSWORD"),
    subscription_key=os.getenv("ERCOT_SUBSCRIPTION_KEY"),
))

ercot = ERCOT(auth=auth)
print("✓ Connected to ERCOT API")

✓ Connected to ERCOT API


## 1. Price Spreads - Battery Arbitrage Analysis

Batteries profit by charging low, discharging high. 

**Note:** DAM pricing endpoints may require specific ERCOT API subscriptions. If pricing data isn't available, we'll use system load as a proxy (load and prices are highly correlated).


In [ ]:
# Fetch DAM settlement point prices for December 2024
# Use a 3-week window for more comprehensive analysis
start_date = datetime(2024, 12, 1)
end_date = datetime(2024, 12, 21)  # 3 weeks of data

print(f"Fetching DAM prices from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Date range: {(end_date - start_date).days + 1} days\n")

# Fetch data for all major ERCOT hubs
# Major hubs: Houston, North, South, West, Pan, BusAvg, HubAvg
target_hubs = ["HB_HOUSTON", "HB_NORTH", "HB_SOUTH", "HB_WEST", "HB_PAN", "HB_BUSAVG", "HB_HUBAVG"]

df_spp_list = []
max_pages_per_hub = 50  # Safety limit per hub

for hub_idx, hub in enumerate(target_hubs, 1):
    print(f"[{hub_idx}/{len(target_hubs)}] Fetching {hub}...", end=" ")
    hub_df_list = []
    page = 1

    while page <= max_pages_per_hub:
        try:
            spp_data = ercot.get_dam_settlement_point_prices(
                delivery_date_from=start_date.strftime("%Y-%m-%d"),
                delivery_date_to=end_date.strftime("%Y-%m-%d"),
                settlement_point=hub,
                size=2000,
                page=page,
            )

            if 'data' in spp_data and 'records' in spp_data['data']:
                records = spp_data['data']['records']
                if not records:
                    break

                fields = spp_data.get('fields', [])
                field_names = [f['name'] for f in fields]
                day_df = pd.DataFrame(records, columns=field_names)
                hub_df_list.append(day_df)

                meta = spp_data.get('_meta', {})
                total_pages = meta.get('totalPages', 1)
                current_page = meta.get('currentPage', page)

                if current_page >= total_pages:
                    break
                page += 1
            else:
                break
        except Exception as e:
            print(f"Error fetching {hub}: {e}")
            break

    if hub_df_list:
        hub_df = pd.concat(hub_df_list, ignore_index=True)
        df_spp_list.append(hub_df)
        print(f"✓ {len(hub_df)} records")
    else:
        print("✗ No data")

df_spp = pd.concat(df_spp_list, ignore_index=True) if df_spp_list else pd.DataFrame()
print(f"\n{'='*60}")
print(f"Total Settlement Point Prices: {len(df_spp):,} records")

# Process the data
if len(df_spp) > 0:
    # Convert date and hour columns
    df_spp['deliveryDate'] = pd.to_datetime(df_spp['deliveryDate'])
    df_spp['hourEnding'] = df_spp['hourEnding'].str.split(':').str[0].astype(int)
    df_spp['settlementPointPrice'] = pd.to_numeric(df_spp['settlementPointPrice'], errors='coerce')

    # Create hub_prices dataframe
    hub_prices = df_spp.copy()
    hub_prices.rename(columns={'settlementPoint': 'busName', 'settlementPointPrice': 'lmp'}, inplace=True)

    # Remove any duplicate records
    hub_prices = hub_prices.drop_duplicates(subset=['deliveryDate', 'hourEnding', 'busName'])

    # Comprehensive statistics
    print(f"\n{'='*60}")
    print("DATA SUMMARY")
    print(f"{'='*60}")
    print(f"Hub price records: {len(hub_prices):,}")
    print(f"Date range: {hub_prices['deliveryDate'].min().strftime('%Y-%m-%d')} to {hub_prices['deliveryDate'].max().strftime('%Y-%m-%d')}")
    print(f"Unique dates: {hub_prices['deliveryDate'].nunique()}")
    print(f"Unique hubs: {hub_prices['busName'].nunique()}")
    print(f"Hubs: {', '.join(sorted(hub_prices['busName'].unique()))}")

    # Hours per day statistics
    hours_per_day = hub_prices.groupby(['deliveryDate', 'busName'])['hourEnding'].nunique()
    print(f"\nHours per day (avg): {hours_per_day.mean():.1f}")
    print(f"Hours per day (min): {hours_per_day.min()}")
    print(f"Hours per day (max): {hours_per_day.max()}")

    # Price statistics by hub
    print(f"\n{'='*60}")
    print("PRICE STATISTICS BY HUB")
    print(f"{'='*60}")
    hub_stats = hub_prices.groupby('busName')['lmp'].agg([
        ('count', 'count'),
        ('mean', 'mean'),
        ('std', 'std'),
        ('min', 'min'),
        ('max', 'max'),
        ('median', 'median'),
    ]).round(2)
    hub_stats['range'] = hub_stats['max'] - hub_stats['min']
    hub_stats['cv'] = (hub_stats['std'] / hub_stats['mean'] * 100).round(2)  # Coefficient of variation
    print(hub_stats.to_string())

    # Daily spread statistics
    daily_spreads = hub_prices.groupby(['deliveryDate', 'busName']).agg(
        min_price=('lmp', 'min'),
        max_price=('lmp', 'max'),
        avg_price=('lmp', 'mean'),
    ).reset_index()
    daily_spreads['spread'] = daily_spreads['max_price'] - daily_spreads['min_price']

    print(f"\n{'='*60}")
    print("DAILY ARBITRAGE SPREAD STATISTICS")
    print(f"{'='*60}")
    spread_stats = daily_spreads.groupby('busName')['spread'].agg([
        ('mean', 'mean'),
        ('std', 'std'),
        ('min', 'min'),
        ('max', 'max'),
        ('median', 'median'),
    ]).round(2)
    print(spread_stats.to_string())

    # Verify data completeness
    if hours_per_day.min() < 20:
        print("\n⚠️ Warning: Some days have fewer than 20 hours. Data may be incomplete.")
else:
    hub_prices = pd.DataFrame()
    daily_spreads = pd.DataFrame()
    print("⚠️ No pricing data available")


### Arbitrage Spread Visualization


In [ ]:
if len(hub_prices) > 0 and 'lmp' in hub_prices.columns and 'deliveryDate' in hub_prices.columns:
    # Calculate daily spreads (reuse from cell 3 if available, otherwise recalculate)
    if 'daily_spreads' not in locals():
        daily_spreads = hub_prices.groupby(['deliveryDate', 'busName']).agg(
            min_price=('lmp', 'min'),
            max_price=('lmp', 'max'),
            avg_price=('lmp', 'mean'),
        ).reset_index()
        daily_spreads['spread'] = daily_spreads['max_price'] - daily_spreads['min_price']

    # Get top hubs by average spread (best arbitrage opportunities)
    top_hubs = daily_spreads.groupby('busName')['spread'].mean().sort_values(ascending=False).head(3).index.tolist()
    main_hub = top_hubs[0] if top_hubs else hub_prices['busName'].iloc[0]

    # Create datetime column
    hub_prices['datetime'] = hub_prices['deliveryDate'] + pd.to_timedelta(hub_prices['hourEnding'], unit='h')

    # Multi-hub comparison visualization
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=(
            'Hourly Prices - Top 3 Hubs (Best Arbitrage)',
            'Daily Arbitrage Spread by Hub',
            'Hub Price Comparison (Last 7 Days)'
        ),
        vertical_spacing=0.1,
        row_heights=[0.4, 0.3, 0.3]
    )

    # Colors for hubs
    hub_colors = {
        'HB_HOUSTON': '#00d4aa',
        'HB_NORTH': '#74b9ff',
        'HB_SOUTH': '#ff6b6b',
        'HB_WEST': '#ffd93d',
        'HB_PAN': '#a29bfe',
        'HB_BUSAVG': '#fd79a8',
        'HB_HUBAVG': '#00cec9',
    }

    # Plot top 3 hubs hourly prices
    for hub in top_hubs[:3]:
        hub_data = hub_prices[hub_prices['busName'] == hub].sort_values(['datetime'])
        color = hub_colors.get(hub, '#888888')
        fig.add_trace(go.Scatter(
            x=hub_data['datetime'], y=hub_data['lmp'],
            mode='lines', name=hub,
            line={"color": color, "width": 1.5},
            legendgroup='prices'
        ), row=1, col=1)

    # Plot daily spreads for all hubs
    for hub in sorted(daily_spreads['busName'].unique()):
        hub_spreads = daily_spreads[daily_spreads['busName'] == hub].sort_values('deliveryDate')
        color = hub_colors.get(hub, '#888888')
        fig.add_trace(go.Scatter(
            x=hub_spreads['deliveryDate'], y=hub_spreads['spread'],
            mode='lines+markers', name=f'{hub} Spread',
            line={"color": color, "width": 1.5},
            marker={"size": 4},
            legendgroup='spreads'
        ), row=2, col=1)

    # Box plot comparing hubs (last 7 days)
    recent_dates = hub_prices['deliveryDate'].max() - pd.Timedelta(days=7)
    recent_prices = hub_prices[hub_prices['deliveryDate'] >= recent_dates]

    for hub in sorted(recent_prices['busName'].unique()):
        hub_data = recent_prices[recent_prices['busName'] == hub]['lmp']
        color = hub_colors.get(hub, '#888888')
        fig.add_trace(go.Box(
            y=hub_data, name=hub,
            marker_color=color,
            boxmean='sd',
            legendgroup='box'
        ), row=3, col=1)

    fig.update_layout(
        height=900, template='plotly_dark', showlegend=True,
        title_text="<b>Multi-Hub Battery Arbitrage Analysis</b><br><sup>Comparing pricing across ERCOT hubs</sup>",
        font={"family": 'Inter, sans-serif'},
        legend={"orientation": 'h', "yanchor": 'bottom', "y": -0.15}
    )
    fig.update_yaxes(title_text="$/MWh", row=1, col=1)
    fig.update_yaxes(title_text="Spread $/MWh", row=2, col=1)
    fig.update_yaxes(title_text="Price $/MWh", row=3, col=1)
    fig.update_xaxes(title_text="Date/Time", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_xaxes(title_text="Hub", row=3, col=1)
    fig.show()

    # Print summary statistics
    print(f"\n{'='*60}")
    print("ARBITRAGE OPPORTUNITY RANKING")
    print(f"{'='*60}")
    spread_ranking = daily_spreads.groupby('busName')['spread'].agg(['mean', 'std', 'max']).sort_values('mean', ascending=False)
    spread_ranking.columns = ['Avg Spread ($/MWh)', 'Std Dev', 'Max Spread ($/MWh)']
    print(spread_ranking.round(2).to_string())
else:
    print("⚠️ No price data available.")


### Hub Correlation Analysis

Understanding price relationships between hubs helps identify arbitrage opportunities and regional price differences.


In [ ]:
if len(hub_prices) > 0 and hub_prices['busName'].nunique() > 1:
    # Create pivot table: dates/hours as index, hubs as columns
    price_pivot = hub_prices.pivot_table(
        index=['deliveryDate', 'hourEnding'],
        columns='busName',
        values='lmp',
        aggfunc='mean'
    )

    # Calculate correlation matrix
    correlation_matrix = price_pivot.corr()

    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.index,
        colorscale=[[0, '#1a1a2e'], [0.5, '#0f3460'], [1, '#00d4aa']],
        colorbar={"title": 'Correlation'},
        text=correlation_matrix.values.round(3),
        texttemplate='%{text}',
        textfont={"size": 10}
    ))

    fig.update_layout(
        title="<b>Hub Price Correlation Matrix</b><br><sup>Higher correlation = similar price movements</sup>",
        xaxis_title="Hub", yaxis_title="Hub",
        template='plotly_dark', height=500,
        font={"family": 'Inter, sans-serif'}
    )
    fig.show()

    # Calculate price differences (spreads between hubs)
    print(f"\n{'='*60}")
    print("HUB-TO-HUB PRICE SPREADS (Average)")
    print(f"{'='*60}")

    hub_means = price_pivot.mean().sort_values(ascending=False)
    print("\nAverage Prices by Hub:")
    for hub, price in hub_means.items():
        print(f"  {hub}: ${price:.2f}/MWh")

    # Find hub pairs with largest average spread
    print(f"\n{'='*60}")
    print("TOP HUB PAIRS FOR ARBITRAGE")
    print(f"{'='*60}")

    spreads = []
    for i, hub1 in enumerate(hub_means.index):
        for hub2 in hub_means.index[i+1:]:
            spread = abs(hub_means[hub1] - hub_means[hub2])
            spreads.append({
                'Hub1': hub1,
                'Hub2': hub2,
                'Avg Spread': spread,
                'Correlation': correlation_matrix.loc[hub1, hub2]
            })

    spread_df = pd.DataFrame(spreads).sort_values('Avg Spread', ascending=False).head(10)
    print(spread_df.to_string(index=False))

    # Hourly spread analysis
    print(f"\n{'='*60}")
    print("HOURLY SPREAD PATTERNS")
    print(f"{'='*60}")

    if len(hub_means) >= 2:
        highest_hub = hub_means.index[0]
        lowest_hub = hub_means.index[-1]

        hourly_spread = price_pivot[highest_hub] - price_pivot[lowest_hub]
        hourly_spread_by_hour = hourly_spread.groupby(level='hourEnding').mean()

        print(f"\nAverage hourly spread: {highest_hub} - {lowest_hub}")
        print(f"  Max spread hour: {hourly_spread_by_hour.idxmax()} ({hourly_spread_by_hour.max():.2f} $/MWh)")
        print(f"  Min spread hour: {hourly_spread_by_hour.idxmin()} ({hourly_spread_by_hour.min():.2f} $/MWh)")
        print(f"  Average spread: {hourly_spread_by_hour.mean():.2f} $/MWh")
else:
    print("⚠️ Need data from multiple hubs for correlation analysis.")


## 2. Optimal Charge/Discharge Windows

When should a battery charge vs discharge? Analyze typical hourly price patterns.


In [ ]:
if len(hub_prices) > 0:
    # Analyze hourly patterns across all hubs
    main_hub = 'HB_HOUSTON' if 'HB_HOUSTON' in hub_prices['busName'].values else hub_prices['busName'].iloc[0]
    main_hub_data = hub_prices[hub_prices['busName'] == main_hub]

    # Average price by hour for main hub
    hourly_pattern = main_hub_data.groupby('hourEnding')['lmp'].agg(['mean', 'std', 'min', 'max']).reset_index()
    avg_price = hourly_pattern['mean'].mean()

    # Classify hours
    hourly_pattern['action'] = hourly_pattern['mean'].apply(
        lambda x: 'CHARGE' if x < avg_price * 0.85 else ('DISCHARGE' if x > avg_price * 1.15 else 'HOLD')
    )

    # Calculate price range (max - min) per hour
    hourly_pattern['price_range'] = hourly_pattern['max'] - hourly_pattern['min']

    colors = {'CHARGE': '#00d4aa', 'DISCHARGE': '#ff6b6b', 'HOLD': '#666666'}

    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(
            f'Optimal Battery Dispatch Schedule - {main_hub}',
            'Hourly Price Volatility (Range)'
        ),
        vertical_spacing=0.15
    )

    # Price bars by action
    for action in ['CHARGE', 'HOLD', 'DISCHARGE']:
        mask = hourly_pattern['action'] == action
        fig.add_trace(go.Bar(
            x=hourly_pattern[mask]['hourEnding'], y=hourly_pattern[mask]['mean'],
            name=action, marker_color=colors[action],
            error_y={"type": 'data', "array": hourly_pattern[mask]['std'], "visible": True, "color": 'rgba(255,255,255,0.3)'},
            legendgroup='price'
        ), row=1, col=1)

    # Price range (volatility)
    fig.add_trace(go.Bar(
        x=hourly_pattern['hourEnding'], y=hourly_pattern['price_range'],
        name='Price Range', marker_color='#ffd93d',
        legendgroup='volatility'
    ), row=2, col=1)

    fig.add_hline(y=avg_price, line_dash="dash", line_color="white",
                  annotation_text=f"Avg: ${avg_price:.2f}", row=1, col=1)

    fig.update_layout(
        title=f"<b>Battery Dispatch Strategy</b><br><sup>{main_hub} - Based on {len(main_hub_data)} hourly observations</sup>",
        xaxis_title="Hour of Day", yaxis_title="Avg LMP ($/MWh)",
        template='plotly_dark', barmode='overlay', height=600,
        legend={"orientation": 'h', "yanchor": 'bottom', "y": -0.15},
        font={"family": 'Inter, sans-serif'}
    )
    fig.update_yaxes(title_text="Price Range ($/MWh)", row=2, col=1)
    fig.update_xaxes(title_text="Hour of Day", row=2, col=1)
    fig.show()

    # Print detailed schedule
    print(f"\n{'='*60}")
    print(f"RECOMMENDED DISPATCH SCHEDULE - {main_hub}")
    print(f"{'='*60}")
    charge_hours = sorted(hourly_pattern[hourly_pattern['action']=='CHARGE']['hourEnding'].tolist())
    discharge_hours = sorted(hourly_pattern[hourly_pattern['action']=='DISCHARGE']['hourEnding'].tolist())
    hold_hours = sorted(hourly_pattern[hourly_pattern['action']=='HOLD']['hourEnding'].tolist())

    print(f"\nCHARGE Hours ({len(charge_hours)}): {charge_hours}")
    print(f"  Avg Price: ${hourly_pattern[hourly_pattern['action']=='CHARGE']['mean'].mean():.2f}/MWh")
    print(f"\nDISCHARGE Hours ({len(discharge_hours)}): {discharge_hours}")
    print(f"  Avg Price: ${hourly_pattern[hourly_pattern['action']=='DISCHARGE']['mean'].mean():.2f}/MWh")
    print(f"\nHOLD Hours ({len(hold_hours)}): {hold_hours}")

    # Best arbitrage windows
    print(f"\n{'='*60}")
    print("BEST ARBITRAGE WINDOWS (Charge Low, Discharge High)")
    print(f"{'='*60}")
    best_charge = hourly_pattern.nsmallest(4, 'mean')[['hourEnding', 'mean']]
    best_discharge = hourly_pattern.nlargest(4, 'mean')[['hourEnding', 'mean']]
    print("\nTop 4 CHARGE Hours:")
    for _, row in best_charge.iterrows():
        print(f"  Hour {row['hourEnding']:2d}: ${row['mean']:.2f}/MWh")
    print("\nTop 4 DISCHARGE Hours:")
    for _, row in best_discharge.iterrows():
        print(f"  Hour {row['hourEnding']:2d}: ${row['mean']:.2f}/MWh")

    potential_spread = best_discharge['mean'].mean() - best_charge['mean'].mean()
    print(f"\nPotential Spread: ${potential_spread:.2f}/MWh")


## 3. System Load by Weather Zone

Understanding load patterns helps retail providers manage portfolios and predict demand.


In [ ]:
# Fetch actual system load for December 2024
load_data = ercot.get_actual_system_load_by_weather_zone(
    operating_day_from=start_date.strftime("%Y-%m-%d"),
    operating_day_to=end_date.strftime("%Y-%m-%d"),
    size=2000,
)

df_load = pd.DataFrame(load_data.get('records', []))
print(f"Load records: {len(df_load)}")

if len(df_load) > 0:
    df_load['datetime'] = pd.to_datetime(df_load['operatingDay'].astype(str) + ' ' + df_load['hourEnding'].astype(str).str.zfill(2) + ':00:00')
    df_load = df_load.sort_values('datetime')

    zones = ['coast', 'east', 'farWest', 'north', 'northC', 'southern', 'southC', 'west']
    available_zones = [z for z in zones if z in df_load.columns]
    for z in available_zones:
        df_load[z] = pd.to_numeric(df_load[z], errors='coerce')

    zone_colors = {
        'coast': '#00d4aa', 'east': '#74b9ff', 'farWest': '#ffd93d',
        'north': '#ff6b6b', 'northC': '#a29bfe', 'southern': '#fd79a8',
        'southC': '#00cec9', 'west': '#e17055'
    }

    fig = go.Figure()
    for zone in available_zones:
        fig.add_trace(go.Scatter(
            x=df_load['datetime'], y=df_load[zone],
            name=zone.upper(), mode='lines', stackgroup='one',
            line={"width": 0.5, "color": zone_colors.get(zone, '#888')},
        ))

    fig.update_layout(
        title="<b>ERCOT System Load by Weather Zone</b>",
        xaxis_title="Date/Time", yaxis_title="Load (MW)",
        template='plotly_dark', height=450, hovermode='x unified',
        legend={"orientation": 'h', "yanchor": 'bottom', "y": 1.02},
        font={"family": 'Inter, sans-serif'}
    )
    fig.show()


## 4. Price Volatility Heatmap

High volatility = opportunity for batteries, risk for retailers.


In [ ]:
if len(hub_prices) > 0:
    hub_prices['dayOfWeek'] = hub_prices['deliveryDate'].dt.dayofweek

    # Calculate volatility by day of week and hour
    volatility = hub_prices.groupby(['dayOfWeek', 'hourEnding'])['lmp'].std().reset_index()
    volatility_pivot = volatility.pivot(index='dayOfWeek', columns='hourEnding', values='lmp')

    day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

    fig = go.Figure(data=go.Heatmap(
        z=volatility_pivot.values,
        x=volatility_pivot.columns,
        y=day_names,
        colorscale=[[0, '#1a1a2e'], [0.5, '#0f3460'], [1, '#ff6b6b']],
        colorbar={"title": 'Std Dev $/MWh'}
    ))

    fig.update_layout(
        title="<b>Price Volatility Heatmap</b><br><sup>When are prices most unpredictable?</sup>",
        xaxis_title="Hour", yaxis_title="Day",
        template='plotly_dark', height=350,
        font={"family": 'Inter, sans-serif'}
    )
    fig.show()


## 5. Battery Revenue Estimation

Estimate potential revenue for a 100 MW / 400 MWh battery system.


In [ ]:
if len(hub_prices) > 0:
    # Battery parameters
    BATTERY_MW = 100   # Power
    BATTERY_MWH = 400  # Energy (4-hour)
    EFFICIENCY = 0.88  # Round-trip

    # Simple arbitrage: charge during 4 lowest hours, discharge during 4 highest
    daily_revenue = []
    for date in hub_prices['deliveryDate'].unique():
        day_prices = hub_prices[hub_prices['deliveryDate'] == date].sort_values('lmp')
        if len(day_prices) >= 8:
            charge_cost = day_prices.head(4)['lmp'].sum() * BATTERY_MW
            discharge_rev = day_prices.tail(4)['lmp'].sum() * BATTERY_MW * EFFICIENCY
            daily_revenue.append({
                'date': date,
                'net_revenue': discharge_rev - charge_cost,
                'avg_charge': day_prices.head(4)['lmp'].mean(),
                'avg_discharge': day_prices.tail(4)['lmp'].mean()
            })

    df_rev = pd.DataFrame(daily_revenue)

    if len(df_rev) > 0:
        fig = make_subplots(rows=2, cols=1,
            subplot_titles=('Daily Net Revenue', 'Charge vs Discharge Prices'),
            vertical_spacing=0.15)

        colors = ['#00d4aa' if x >= 0 else '#ff6b6b' for x in df_rev['net_revenue']]
        fig.add_trace(go.Bar(x=df_rev['date'], y=df_rev['net_revenue'], marker_color=colors), row=1, col=1)
        fig.add_trace(go.Scatter(x=df_rev['date'], y=df_rev['avg_charge'], name='Charge', line={"color": '#00d4aa'}), row=2, col=1)
        fig.add_trace(go.Scatter(x=df_rev['date'], y=df_rev['avg_discharge'], name='Discharge', line={"color": '#ff6b6b'}), row=2, col=1)

        fig.update_layout(
            title=f"<b>Battery Revenue</b><br><sup>{BATTERY_MW}MW/{BATTERY_MWH}MWh, {EFFICIENCY*100:.0f}% efficiency</sup>",
            template='plotly_dark', height=500, showlegend=True,
            legend={"orientation": 'h', "yanchor": 'bottom', "y": 1.02},
            font={"family": 'Inter, sans-serif'}
        )
        fig.update_yaxes(title_text="$", row=1, col=1)
        fig.update_yaxes(title_text="$/MWh", row=2, col=1)
        fig.show()

        print(f"\n💰 Revenue Summary ({BATTERY_MW}MW / {BATTERY_MWH}MWh)")
        print(f"   Total: ${df_rev['net_revenue'].sum():,.0f}")
        print(f"   Daily Avg: ${df_rev['net_revenue'].mean():,.0f}")
        print(f"   Best Day: ${df_rev['net_revenue'].max():,.0f}")
        print(f"   Annualized: ${df_rev['net_revenue'].mean() * 365:,.0f}")


## 6. Renewable Generation Patterns

Solar and wind patterns drive price dynamics. Critical for battery timing and retail hedging.


In [ ]:
# Fetch solar and wind data
solar_data = ercot.get_spp_hourly_average_actual_forecast(size=500)
wind_data = ercot.get_wpp_hourly_average_actual_forecast(size=500)

df_solar = pd.DataFrame(solar_data.get('records', []))
df_wind = pd.DataFrame(wind_data.get('records', []))

print(f"Solar: {len(df_solar)} records, Wind: {len(df_wind)} records")

if len(df_solar) > 0 and len(df_wind) > 0:
    # Process solar
    if 'operatingDay' in df_solar.columns and 'hourEnding' in df_solar.columns:
        df_solar['datetime'] = pd.to_datetime(df_solar['operatingDay'].astype(str) + ' ' + df_solar['hourEnding'].astype(str).str.zfill(2) + ':00:00')
        df_solar = df_solar.sort_values('datetime')

    # Process wind
    if 'operatingDay' in df_wind.columns and 'hourEnding' in df_wind.columns:
        df_wind['datetime'] = pd.to_datetime(df_wind['operatingDay'].astype(str) + ' ' + df_wind['hourEnding'].astype(str).str.zfill(2) + ':00:00')
        df_wind = df_wind.sort_values('datetime')

    fig = make_subplots(rows=2, cols=1,
        subplot_titles=('Solar Generation (MW)', 'Wind Generation (MW)'),
        vertical_spacing=0.1)

    if 'systemWide' in df_solar.columns:
        fig.add_trace(go.Scatter(
            x=df_solar['datetime'], y=pd.to_numeric(df_solar['systemWide'], errors='coerce'),
            name='Solar', line={"color": '#ffd93d', "width": 2},
            fill='tozeroy', fillcolor='rgba(255, 217, 61, 0.3)'
        ), row=1, col=1)

    if 'systemWide' in df_wind.columns:
        fig.add_trace(go.Scatter(
            x=df_wind['datetime'], y=pd.to_numeric(df_wind['systemWide'], errors='coerce'),
            name='Wind', line={"color": '#74b9ff', "width": 2},
            fill='tozeroy', fillcolor='rgba(116, 185, 255, 0.3)'
        ), row=2, col=1)

    fig.update_layout(
        height=500, template='plotly_dark',
        title_text="<b>Renewable Generation</b>",
        font={"family": 'Inter, sans-serif'},
        showlegend=False
    )
    fig.show()


## Summary

This notebook demonstrated:

**Battery Storage:**
- Price spread analysis for arbitrage revenue
- Optimal charge/discharge scheduling
- Revenue modeling with efficiency losses

**Retail Electricity:**
- Load patterns by weather zone
- Price volatility for risk assessment
- Renewable generation patterns

All powered by the Tiny Grid SDK with just a few lines of code.
